In [ ]:
import nannyml as nml
from IPython.display import display

reference_df, analysis_df, analysis_target_df = nml.load_synthetic_car_loan_dataset()

analysis_target_df.head(3)

,repaid
0,1
1,1
2,1


In [ ]:
print(analysis_target_df.head(3).to_markdown(tablefmt="grid"))

+----+----------+
|    |   repaid |
+====+==========+
|  0 |        1 |
+----+----------+
|  1 |        1 |
+----+----------+
|  2 |        1 |
+----+----------+


In [ ]:
analysis_with_targets = analysis_df.merge(analysis_target_df, left_index=True, right_index=True)

display(analysis_with_targets.head(3))

,car_value,salary_range,debt_to_income_ratio,loan_length,repaid_loan_on_prev_car,size_of_downpayment,driver_tenure,timestamp,y_pred_proba,y_pred,repaid
0,12638.0,0 - 20K €,0.487926,21.0,False,10%,4.224628,2018-10-30 18:00:00.000,0.99,1,1
1,52425.0,20K - 20K €,0.672183,20.0,False,40%,4.963103,2018-10-30 18:08:43.152,0.98,1,1
2,20369.0,40K - 60K €,0.703090,19.0,True,40%,4.588951,2018-10-30 18:17:26.304,0.98,1,1


In [ ]:
print(analysis_with_targets.head(3).to_markdown(tablefmt="grid"))

+----+-------------+----------------+------------------------+---------------+---------------------------+-----------------------+-----------------+-------------------------+----------------+----------+----------+
|    |   car_value | salary_range   |   debt_to_income_ratio |   loan_length | repaid_loan_on_prev_car   | size_of_downpayment   |   driver_tenure | timestamp               |   y_pred_proba |   y_pred |   repaid |
+====+=============+================+========================+===============+===========================+=======================+=================+=========================+================+==========+==========+
|  0 |       12638 | 0 - 20K €      |               0.487926 |            21 | False                     | 10%                   |         4.22463 | 2018-10-30 18:00:00.000 |           0.99 |        1 |        1 |
+----+-------------+----------------+------------------------+---------------+---------------------------+-----------------------+--------------

In [ ]:
# Estimate performance without targets
estimator = nml.CBPE(
    y_pred_proba='y_pred_proba',
    y_pred='y_pred',
    y_true='repaid',
    timestamp_column_name='timestamp',
    metrics=['roc_auc'],
    chunk_size=5000,
    problem_type='classification_binary',
)


In [ ]:
estimator.fit(reference_df)

results = estimator.estimate(analysis_df)

display(results.filter(period='analysis').to_df())

chunk                                                        \
             key chunk_index start_index end_index          start_date   
0       [0:4999]           0           0      4999 2018-10-30 18:00:00   
1    [5000:9999]           1        5000      9999 2018-11-30 00:36:00   
2  [10000:14999]           2       10000     14999 2018-12-30 07:12:00   
3  [15000:19999]           3       15000     19999 2019-01-29 13:48:00   
4  [20000:24999]           4       20000     24999 2019-02-28 20:24:00   
5  [25000:29999]           5       25000     29999 2019-03-31 03:00:00   
6  [30000:34999]           6       30000     34999 2019-04-30 09:36:00   
7  [35000:39999]           7       35000     39999 2019-05-30 16:12:00   
8  [40000:44999]           8       40000     44999 2019-06-29 22:48:00   
9  [45000:49999]           9       45000     49999 2019-07-30 05:24:00   

                                      roc_auc                          \
                 end_date    period     value sampling_error realized   
0 2018-11-30 00:27:16.848  analysis  0.968631       0.001811      NaN   
1 2018-12-30 07:03:16.848  analysis  0.969044       0.001811      NaN   
2 2019-01-29 13:39:16.848  analysis  0.969444       0.001811      NaN   
3 2019-02-28 20:15:16.848  analysis  0.969047       0.001811      NaN   
4 2019-03-31 02:51:16.848  analysis  0.968873       0.001811      NaN   
5 2019-04-30 09:27:16.848  analysis  0.960478       0.001811      NaN   
6 2019-05-30 16:03:16.848  analysis  0.961134       0.001811      NaN   
7 2019-06-29 22:39:16.848  analysis  0.960536       0.001811      NaN   
8 2019-07-30 05:15:16.848  analysis  0.961869       0.001811      NaN   
9 2019-08-29 11:51:16.848  analysis  0.960537       0.001811      NaN   

                                                                       \
  upper_confidence_boundary lower_confidence_boundary upper_threshold   
0                  0.974063                  0.963198         0.97866   
1                  0.974476                  0.963612         0.97866   
2                  0.974876                  0.964012         0.97866   
3                  0.974479                  0.963615         0.97866   
4                  0.974305                  0.963441         0.97866   
5                  0.965910                  0.955046         0.97866   
6                  0.966566                  0.955701         0.97866   
7                  0.965968                  0.955104         0.97866   
8                  0.967301                  0.956437         0.97866   
9                  0.965969                  0.955104         0.97866   

                          
  lower_threshold  alert  
0        0.963317  False  
1        0.963317  False  
2        0.963317  False  
3        0.963317  False  
4        0.963317  False  
5        0.963317   True  
6        0.963317   True  
7        0.963317   True  
8        0.963317   True  
9        0.963317   True

In [ ]:
from docs.utils import print_multi_index_markdown
print_multi_index_markdown(results.filter(period='analysis').to_df())

+----+---------------+-----------------+-----------------+---------------+---------------------+----------------------------+------------+-------------+--------------------+--------------+-------------------------------+-------------------------------+---------------------+---------------------+-----------+
|    | | chunk       |                 |                 |               |                     |                            |            | | roc_auc   |                    |              |                               |                               |                     |                     |           |
|    | | key         | | chunk_index   | | start_index   | | end_index   | | start_date        | | end_date                 | | period   | | value     | | sampling_error   | | realized   | | upper_confidence_boundary   | | lower_confidence_boundary   | | upper_threshold   | | lower_threshold   | | alert   |
+====+===============+=================+=================+===============

In [ ]:
# Calculate realized performance using targets
calculator = nml.PerformanceCalculator(
    y_pred_proba='y_pred_proba',
    y_pred='y_pred',
    y_true='repaid',
    timestamp_column_name='timestamp',
    metrics=['roc_auc'],
    chunk_size=5000,
    problem_type='classification_binary',
).fit(reference_df)
realized_results = calculator.calculate(analysis_with_targets)
display(realized_results.filter(period='analysis').to_df())

chunk                                                        \
             key chunk_index start_index end_index          start_date   
0       [0:4999]           0           0      4999 2018-10-30 18:00:00   
1    [5000:9999]           1        5000      9999 2018-11-30 00:36:00   
2  [10000:14999]           2       10000     14999 2018-12-30 07:12:00   
3  [15000:19999]           3       15000     19999 2019-01-29 13:48:00   
4  [20000:24999]           4       20000     24999 2019-02-28 20:24:00   
5  [25000:29999]           5       25000     29999 2019-03-31 03:00:00   
6  [30000:34999]           6       30000     34999 2019-04-30 09:36:00   
7  [35000:39999]           7       35000     39999 2019-05-30 16:12:00   
8  [40000:44999]           8       40000     44999 2019-06-29 22:48:00   
9  [45000:49999]           9       45000     49999 2019-07-30 05:24:00   

                                                                roc_auc  \
                 end_date    period targets_missing_rate sampling_error   
0 2018-11-30 00:27:16.848  analysis                  0.0       0.001811   
1 2018-12-30 07:03:16.848  analysis                  0.0       0.001811   
2 2019-01-29 13:39:16.848  analysis                  0.0       0.001811   
3 2019-02-28 20:15:16.848  analysis                  0.0       0.001811   
4 2019-03-31 02:51:16.848  analysis                  0.0       0.001811   
5 2019-04-30 09:27:16.848  analysis                  0.0       0.001811   
6 2019-05-30 16:03:16.848  analysis                  0.0       0.001811   
7 2019-06-29 22:39:16.848  analysis                  0.0       0.001811   
8 2019-07-30 05:15:16.848  analysis                  0.0       0.001811   
9 2019-08-29 11:51:16.848  analysis                  0.0       0.001811   

                                                    
      value upper_threshold lower_threshold  alert  
0  0.970962         0.97866        0.963317  False  
1  0.970248         0.97866        0.963317  False  
2  0.976282         0.97866        0.963317  False  
3  0.967721         0.97866        0.963317  False  
4  0.969886         0.97866        0.963317  False  
5  0.960050         0.97866        0.963317   True  
6  0.958530         0.97866        0.963317   True  
7  0.959041         0.97866        0.963317   True  
8  0.963094         0.97866        0.963317   True  
9  0.957556         0.97866        0.963317   True

In [ ]:
print_multi_index_markdown(realized_results.filter(period='analysis').to_df())

+----+---------------+-----------------+-----------------+---------------+---------------------+----------------------------+------------+--------------------------+--------------------+-----------+---------------------+---------------------+-----------+
|    | | chunk       |                 |                 |               |                     |                            |            |                          | | roc_auc          |           |                     |                     |           |
|    | | key         | | chunk_index   | | start_index   | | end_index   | | start_date        | | end_date                 | | period   | | targets_missing_rate   | | sampling_error   | | value   | | upper_threshold   | | lower_threshold   | | alert   |
+====+===============+=================+=================+===============+=====================+============================+============+==========================+====================+===========+=====================+===============

In [ ]:
# Show comparison plot
results.filter(period='analysis').compare(realized_results).plot().show()

In [ ]:
results.filter(period='analysis').compare(realized_results).plot().write_image('../_static/tutorials/estimated_and_realized_performance/comparison_plot.svg', width=1500)
